<a href="https://colab.research.google.com/github/sreeman-11021996/Fashion-Recommendation-system/blob/main/train_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install import_ipynb
import import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d paramaggarwal/fashion-product-images-small

fashion-product-images-small.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import zipfile
zip_ref = zipfile.ZipFile("/content/fashion-product-images-small.zip", "r")
zip_ref.extractall("/content")
zip_ref.close()

In [5]:
import tensorflow
from keras.models import Model

import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import Model

In [6]:
from utils import *
from dataset import train_val_split,FashionDataset

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from dataset.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
#Loading the labels and removing missing images:
data=pd.read_csv('/content/styles.csv',usecols=["id","gender","masterCategory",
                                                 "subCategory","articleType",
                                                 "baseColour","season","usage"])

In [8]:
# training and validation dataset
dataset = FashionDataset(data,labels=["gender","masterCategory","subCategory"])

[INFO]: Checking if all images are present


44446it [00:03, 11727.12it/s]


[INFO]: Dropping indices: [6697, 16207, 32324, 36399, 40022]


In [9]:
dataset.df.head()

,id,classes
0,15970.jpg,"(Men, Apparel, Topwear)"
1,39386.jpg,"(Men, Apparel, Bottomwear)"
2,59263.jpg,"(Women, Accessories, Watches)"
3,21379.jpg,"(Men, Apparel, Bottomwear)"
4,53759.jpg,"(Men, Apparel, Topwear)"


In [10]:
train_data, val_data = train_val_split(dataset.df)
print(f"[INFO]: Number of training sampels: {len(train_data)}")
print(f"[INFO]: Number of validation sampels: {len(val_data)}")

[INFO]: Number of training sampels: 39996
[INFO]: Number of validation sampels: 4444


In [11]:
train_data.tail()

,index,id,classes
39991,39991,24738.jpg,"(Women, Accessories, Bags)"
39992,39992,46788.jpg,"(Women, Personal Care, Fragrance)"
39993,39993,40954.jpg,"(Boys, Apparel, Topwear)"
39994,39994,15127.jpg,"(Men, Apparel, Bottomwear)"
39995,39995,53303.jpg,"(Men, Apparel, Topwear)"


In [24]:
batch_size = 64
# training and validation data loader
train_generator = dataset.datagen.flow_from_dataframe(
    train_data, batch_size=batch_size,directory='./images/',
    x_col='id',y_col='index',class_mode='other',color_mode="rgb",
    target_size=(224, 224),shuffle=True
)
val_generator = dataset.datagen.flow_from_dataframe(
    val_data, batch_size=batch_size,directory='./images/',
    x_col='id',y_col='index',class_mode='other',color_mode="rgb",
    target_size=(224, 224),shuffle=True
)

Found 39996 validated image filenames.
Found 4444 validated image filenames.


In [25]:
from sklearn.preprocessing import MultiLabelBinarizer
# Fit a MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(dataset.df['classes'].values.tolist())

MultiLabelBinarizer()

In [26]:
len(mlb.classes_)

56

In [28]:
train_multilabel_generator = multilabel_flow_from_dataframe(train_generator, mlb, train_data)
val_multilabel_generator = multilabel_flow_from_dataframe(val_generator, mlb, val_data)

In [29]:
# Load model architecture
model_file = open("./myModel_arc.json","r")
loaded_json_model = model_file.read()
model_file.close()
loaded_model = model_from_json(loaded_json_model)

In [30]:
model = Model(inputs=loaded_model.input,outputs=loaded_model.layers[len(loaded_model.layers)-2].output)

In [31]:
num_classes = len(mlb.classes_)

In [32]:
final_model = tensorflow.keras.Sequential([
    model,
    tensorflow.keras.layers.Dense(num_classes,activation="softmax",name="fc"+str(num_classes))
])

In [33]:
lr = 0.001
optim = tensorflow.keras.optimizers.RMSprop(learning_rate=lr)
epochs = 4
loss_func = "binary_crossentropy"

In [34]:
final_model.compile(optimizer=optim,loss = loss_func,metrics=["accuracy"])

In [23]:
history = final_model.fit(train_multilabel_generator,epochs=epochs,validation_data=val_multilabel_generator)

Epoch 1/20
     11/Unknown - 269s 25s/step - loss: 0.4356 - accuracy: 0.1761

KeyboardInterrupt: ignored